In [ ]:
%pip install datasets transformers evaluate scikit-learn 'transformers[torch]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.

In [ ]:
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate
import argparse
import torch
import torch.nn.functional as F
from sklearn.utils.class_weight import compute_class_weight


In [ ]:
def get_special_tokens():
    #[SEP] is already a special token in BERT, these are additional special tokens
    return ['[USER]', '[LINK]', '[EMAIL]', '[IMAGE]', '[QUOTED_TEXT]', '[EMPTY]', '[HTML]']

In [ ]:
token = #REDACTED

In [ ]:

def train(input_csv: str, category_file: str, checkpoint: str = None):

    with open(category_file, 'r') as f:
        categories = f.readlines()
        num_labels = len(categories)

    dataset = load_dataset('csv', data_files=input_csv, split='train')
    dataset = dataset.train_test_split(test_size=0.2)

    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    special_tokens = get_special_tokens()
    tokenizer.add_tokens(special_tokens)
    def tokenize_function(examples):
        return tokenizer(examples['text'], padding="max_length", truncation=True)
    tokenized_dataset = dataset.map(tokenize_function, batched=True)

    train_dataset = tokenized_dataset["train"].shuffle(seed=42)#.select(range(1000))
    eval_dataset = tokenized_dataset["test"].shuffle(seed=42)#.select(range(1000))

    model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=num_labels)
    model.resize_token_embeddings(len(tokenizer))
    training_args = TrainingArguments(output_dir="test_trainer",
                                      evaluation_strategy="epoch",
                                      save_strategy="epoch",
                                      per_device_train_batch_size=32,
                                      per_device_eval_batch_size=8,
                                      num_train_epochs=10,
                                      )
    metric = evaluate.load("f1")

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels, average="micro")

    device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
    model.to(device)
    #class_weights_tensor = compute_class_weights(train_dataset)
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )
    # trainer = WeightedTrainer(
    #     model=model,
    #     args=training_args,
    #     train_dataset=train_dataset,
    #     eval_dataset=eval_dataset,
    #     compute_metrics=compute_metrics,
    #     class_weights=class_weights_tensor
    # )

    trainer.train(resume_from_checkpoint=checkpoint)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%pip install accelerate -U

In [ ]:
%pip install transformers==4.30

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.0
    Uninstalling transformers-4.40.0:
      Successfully uninstalled transformers-4.40.0


In [ ]:
#login to hugging face
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
train('drive/MyDrive/cleaned_all.csv', 'training_categories')

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Map:   0%|          | 0/15144 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1
1,0.229800,0.154303,0.952456
2,0.122400,0.108872,0.967248
3,0.085600,0.104100,0.968700
4,0.070700,0.082483,0.974908
5,0.061300,0.093874,0.976096
6,0.046700,0.081851,0.977945
7,0.043800,0.093493,0.976162
8,0.033300,0.083200,0.978407
9,0.030100,0.092263,0.978539
10,0.025800,0.108456,0.978143


In [ ]:
!zip -r /content/training_info.zip /content/test_trainer

zip I/O error: Operation not supported
zip error: Could not create output file (/content/drive/training_info.zip)


In [ ]:
!cp /content/training_info.zip /gdrive/MyDrive/

cp: cannot create regular file '/gdrive/MyDrive/': No such file or directory


In [ ]:
#download /content/training_info.zip

from google.colab import files
files.download('/content/training_info.zip')

In [1]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1_metric = load_metric("f1")
    acc_metric = load_metric("accuracy")
    per_class_f1 = f1_metric.compute(predictions=predictions, references=labels, average=None)['f1']
    return {
        "f1": f1_metric.compute(predictions=predictions, references=labels, average="micro")['f1'],
        "accuracy": acc_metric.compute(predictions=predictions, references=labels)['accuracy'],
        "class_0_f1": per_class_f1[0],
        "class_1_f1": per_class_f1[1],
        "class_2_f1": per_class_f1[2],
        "class_3_f1": per_class_f1[3],
        "class_4_f1": per_class_f1[4],
        "class_5_f1": per_class_f1[5],
        "class_6_f1": per_class_f1[6],
        "class_7_f1": per_class_f1[7],
    }

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
import os
from datasets import load_metric

In [ ]:
dataset = load_dataset('csv', data_files='drive/MyDrive/cleaned_all.csv', split='train')
dataset = dataset.train_test_split(test_size=0.05)
eval_dataset = dataset["test"]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
special_tokens = get_special_tokens()
tokenizer.add_tokens(special_tokens)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3786 [00:00<?, ? examples/s]

In [ ]:
results = []
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
checkpoints_dir = "test_trainer"

for checkpoint in sorted(os.listdir("test_trainer")):
    if checkpoint.startswith("checkpoint-"):
        model_path = os.path.join(checkpoints_dir, checkpoint)
        print(model_path)
        model = AutoModelForSequenceClassification.from_pretrained(model_path)
        model.to(device)

        trainer = Trainer(
            model=model,
            args=TrainingArguments(output_dir="temp_dir"),  # Temp dir for evaluation
            eval_dataset=tokenized_eval_dataset,
            compute_metrics=compute_metrics
        )

        eval_results = trainer.evaluate()
        print(eval_results)
        results.append({
            "checkpoint": checkpoint,
            "f1": eval_results["eval_f1"],
            "accuracy": eval_results["eval_accuracy"],
            "class_0_f1": eval_results["eval_class_0_f1"],
            "class_1_f1": eval_results["eval_class_1_f1"],
            "class_2_f1": eval_results["eval_class_2_f1"],
            "class_3_f1": eval_results["eval_class_3_f1"],
            "class_4_f1": eval_results["eval_class_4_f1"],
            "class_5_f1": eval_results["eval_class_5_f1"],
            "class_6_f1": eval_results["eval_class_6_f1"],
            "class_7_f1": eval_results["eval_class_7_f1"],
        })

# Convert results to DataFrame and save as CSV
results_df = pd.DataFrame(results)
results_df.to_csv("model_evaluation_results.csv", index=False)

test_trainer/checkpoint-11358


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next majo

{'eval_loss': 0.04013960808515549, 'eval_f1': 0.9838880084521923, 'eval_accuracy': 0.9838880084521923, 'eval_class_0_f1': 0.9893121403124144, 'eval_class_1_f1': 0.9620817843866171, 'eval_class_2_f1': 0.9686746987951808, 'eval_class_3_f1': 0.9887640449438202, 'eval_class_4_f1': 0.981636060100167, 'eval_class_5_f1': 0.993939393939394, 'eval_class_6_f1': 1.0, 'eval_class_7_f1': 1.0, 'eval_runtime': 32.8193, 'eval_samples_per_second': 115.359, 'eval_steps_per_second': 14.443}
test_trainer/checkpoint-13251


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next majo

{'eval_loss': 0.040262091904878616, 'eval_f1': 0.9833597464342314, 'eval_accuracy': 0.9833597464342314, 'eval_class_0_f1': 0.989849108367627, 'eval_class_1_f1': 0.9585798816568046, 'eval_class_2_f1': 0.9731051344743276, 'eval_class_3_f1': 0.9887640449438202, 'eval_class_4_f1': 0.9750415973377704, 'eval_class_5_f1': 0.994954591321897, 'eval_class_6_f1': 1.0, 'eval_class_7_f1': 1.0, 'eval_runtime': 32.7845, 'eval_samples_per_second': 115.482, 'eval_steps_per_second': 14.458}
test_trainer/checkpoint-15144


...

{'eval_loss': 0.07093462347984314, 'eval_f1': 0.9762282091917591, 'eval_accuracy': 0.9762282091917591, 'eval_class_0_f1': 0.9873417721518988, 'eval_class_1_f1': 0.9485458612975392, 'eval_class_2_f1': 0.9692671394799054, 'eval_class_3_f1': 0.9473684210526316, 'eval_class_4_f1': 0.954248366013072, 'eval_class_5_f1': 0.98989898989899, 'eval_class_6_f1': 0.9820359281437125, 'eval_class_7_f1': 0.9914529914529915, 'eval_runtime': 32.7568, 'eval_samples_per_second': 115.579, 'eval_steps_per_second': 14.47}
test_trainer/checkpoint-7572


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next majo

{'eval_loss': 0.05687284842133522, 'eval_f1': 0.9801901743264659, 'eval_accuracy': 0.9801901743264659, 'eval_class_0_f1': 0.9882030178326474, 'eval_class_1_f1': 0.9563286454478165, 'eval_class_2_f1': 0.9611650485436893, 'eval_class_3_f1': 0.9888888888888889, 'eval_class_4_f1': 0.9698996655518394, 'eval_class_5_f1': 0.9888776541961577, 'eval_class_6_f1': 1.0, 'eval_class_7_f1': 0.9957081545064378, 'eval_runtime': 32.7985, 'eval_samples_per_second': 115.432, 'eval_steps_per_second': 14.452}
test_trainer/checkpoint-9465


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next majo

{'eval_loss': 0.05651785433292389, 'eval_f1': 0.9807184363444268, 'eval_accuracy': 0.9807184363444268, 'eval_class_0_f1': 0.9879584017515053, 'eval_class_1_f1': 0.957683741648107, 'eval_class_2_f1': 0.9705882352941176, 'eval_class_3_f1': 0.9887640449438202, 'eval_class_4_f1': 0.9647058823529412, 'eval_class_5_f1': 0.9909365558912387, 'eval_class_6_f1': 0.993939393939394, 'eval_class_7_f1': 1.0, 'eval_runtime': 32.815, 'eval_samples_per_second': 115.374, 'eval_steps_per_second': 14.445}


In [ ]:
print(results_df)

         checkpoint        f1  accuracy  class_0_f1  class_1_f1  class_2_f1  \
0  checkpoint-11358  0.983888  0.983888    0.989312    0.962082    0.968675   
1  checkpoint-13251  0.983360  0.983360    0.989849    0.958580    0.973105   
2  checkpoint-15144  0.983360  0.983360    0.989855    0.960000    0.970588   
3  checkpoint-17037  0.985209  0.985209    0.990398    0.962194    0.978102   
4   checkpoint-1893  0.957739  0.957739    0.980381    0.930131    0.913580   
5  checkpoint-18930  0.984945  0.984945    0.990121    0.961481    0.978102   
6   checkpoint-3786  0.975172  0.975172    0.986505    0.951111    0.950820   
7   checkpoint-5679  0.976228  0.976228    0.987342    0.948546    0.969267   
8   checkpoint-7572  0.980190  0.980190    0.988203    0.956329    0.961165   
9   checkpoint-9465  0.980718  0.980718    0.987958    0.957684    0.970588   

   class_3_f1  class_4_f1  class_5_f1  class_6_f1  class_7_f1  
0    0.988764    0.981636    0.993939    1.000000    1.000000  
1 